In [1]:
import numpy as np
import pandas as pd
import sqlite3
import category_encoders as ce
import re

from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import FunctionTransformer

conn = sqlite3.connect('db/wine_data.sqlite')
c = conn.cursor

In [2]:
#create Dataframe
df = pd.read_sql("select country  \
                 ,description     \
                 ,rating          \
                 ,price           \
                 ,province        \
                 ,title           \
                 ,winery from wine_data where variety = 'Chardonnay'", conn)
#df.head(2)

In [3]:
year_column = 'title'
word_count_column = 'description'

def extract_year(dataframe):
    global year_column
    years = dataframe[year_column]
    #years.reset_index(inplace=False)
    #years.fillna("", inplace=True)
    l = []
    i = 0 
    for year in range(len(dataframe)):
        temp = re.findall(r'\d+', years[i]) 
        res = list(map(int, temp)) 
        try: 
            if len(str(res[0])) == 4:
                l.append(res[0])
            elif len(str(res[0])) != 4:
                l.append(0)
        except:
            l.append(0)
        #print(res[0])
        i+=1
    dataframe['year'] = l

    return dataframe
#df = extract_year(df)


In [4]:
def word_count(dataframe):
    global word_count_column
    dataframe['word_count'] = dataframe[word_count_column].apply(lambda word: len(str(word).split(" ")))
    return dataframe
# df = word_count(df)
# df.head(3)

In [5]:
# encoder = ce.JamesSteinEncoder(cols=[...]) --maybe (best score)
# encoder = ce.LeaveOneOutEncoder(cols=[...]) --maybe
# encoder = ce.MEstimateEncoder(cols=[...]) --maybe (good)
# encoder = ce.OrdinalEncoder(cols=[...]) --maybe
# encoder = ce.TargetEncoder(cols=[...]) --maybe

year_column = 'title'
word_count_column = 'description'
category_columns = ['country','province','title','winery']
target = 'price'
combine_text = ['country','province','title','winery', 'description']
numeric= ['price', 'year', 'word_count','country','province','title','winery']

def category_encode(dataframe):
    global category_columns
    global category_target
    x = dataframe[category_columns]
    y = dataframe[target]
    ce_ord = ce.OrdinalEncoder(cols=category_columns)
    dataframe[category_columns] = ce_ord.fit_transform(x, y)
    return dataframe

# df = category_encode(df)
# df.head()

In [6]:
get_year = FunctionTransformer(extract_year, validate=False)
get_word_count = FunctionTransformer(word_count, validate=False)
get_encoded_text = FunctionTransformer(category_encode, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[numeric], validate=False)
def reset_index(dataframe):
    dataframe = dataframe.reset_index(inplace = False)
    return dataframe
get_reset_index = FunctionTransformer(reset_index, validate=False)

In [7]:
from sklearn import ensemble 
model = ensemble.GradientBoostingRegressor(
    n_estimators = 100, #how many decision trees to build
    learning_rate = 0.5, #controls rate at which additional decision trees influes overall prediction
    max_depth = 6, 
    min_samples_split = 21,
    min_samples_leaf = 19, 
    max_features = 0.9,
    loss = 'huber'
)

pl = Pipeline(memory=None,
    steps=[
        ('reset_index', get_reset_index),
        ('year', get_year),
        ('word_count', get_word_count),
        ('encode', get_encoded_text),
        ('selector', get_numeric_data),
        ('model', model)
    ], verbose=False)


In [8]:
features = df.drop(['rating'], axis=1)

X = features
y = df['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y
                                                   , test_size = .3
                                                   #, stratify=y
                                                   )
pl.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('reset_index',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function reset_index at 0x0000019074928438>,
                                     inv_kw_args=None, inverse_func=None,
                                     kw_args=None, pass_y='deprecated',
                                     validate=False)),
                ('year',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function extract_year at 0x0000019075043EE8>,
                                     i...
                                           init=None, learning_rate=0.5,
                                           loss='huber', max_depth=6,
                                           max_features=0.9,
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
       

In [9]:
pl.score(X_test, y_test)

0.31234110635900303